# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [14]:
# import libraries
import pandas as pd
import numpy as np
import nltk
import string
from nltk import word_tokenize
from nltk.corpus import stopwords
from sqlalchemy import create_engine
import sqlite3
from sklearn import model_selection
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.multioutput import MultiOutputClassifier

In [83]:
# load data from database
engine = create_engine('sqlite:///../data/DisasterResponse.db')

connection = engine.connect()
query = "SELECT * FROM DisasterMessages"
df = pd.read_sql(query, connection)
connection.close()

df.drop(columns="child_alone",inplace=True)
df.dtypes

id                         int64
message                   object
genre                     object
related                    int64
request                    int64
offer                      int64
aid_related                int64
medical_help               int64
medical_products           int64
search_and_rescue          int64
security                   int64
military                   int64
water                      int64
food                       int64
shelter                    int64
clothing                   int64
money                      int64
missing_people             int64
refugees                   int64
death                      int64
other_aid                  int64
infrastructure_related     int64
transport                  int64
buildings                  int64
electricity                int64
tools                      int64
hospitals                  int64
shops                      int64
aid_centers                int64
other_infrastructure       int64
weather_re

In [86]:
# Select X and Y for analysis

X = df["message"].values

N_CATEGORIES = 34 #number of category columns
Y = df.iloc[:,-N_CATEGORIES:].values

### 2. Write a tokenization function to process your text data

In [87]:
def tokenize(text):
    # Bring to lowercase, remove stopwords and punctuation
    stop = set(stopwords.words('english') + list(string.punctuation))
    nltk_tokens = nltk.word_tokenize(text.lower())
    tokens = [w for w in nltk_tokens if not w in stop]
    return(tokens)


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [88]:
tfidf_vectorizer = TfidfVectorizer(tokenizer=tokenize)
mo = MultiOutputClassifier(LogisticRegression(solver="liblinear"))
pipeline = Pipeline([('tfidf', tfidf_vectorizer), ('mo', mo)])

### 4. Train pipeline
- Split data into train and test sets


In [89]:
X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(X, Y, test_size=0.2, random_state=42)

In [90]:
for i in range(0,N_CATEGORIES):
    print(i, np.unique(Y[:,i], return_counts=True))

0 (array([0, 1], dtype=int64), array([21742,  4474], dtype=int64))
1 (array([0, 1], dtype=int64), array([26098,   118], dtype=int64))
2 (array([0, 1], dtype=int64), array([15356, 10860], dtype=int64))
3 (array([0, 1], dtype=int64), array([24132,  2084], dtype=int64))
4 (array([0, 1], dtype=int64), array([24903,  1313], dtype=int64))
5 (array([0, 1], dtype=int64), array([25492,   724], dtype=int64))
6 (array([0, 1], dtype=int64), array([25745,   471], dtype=int64))
7 (array([0, 1], dtype=int64), array([25356,   860], dtype=int64))
8 (array([0, 1], dtype=int64), array([24544,  1672], dtype=int64))
9 (array([0, 1], dtype=int64), array([23293,  2923], dtype=int64))
10 (array([0, 1], dtype=int64), array([23902,  2314], dtype=int64))
11 (array([0, 1], dtype=int64), array([25811,   405], dtype=int64))
12 (array([0, 1], dtype=int64), array([25612,   604], dtype=int64))
13 (array([0, 1], dtype=int64), array([25918,   298], dtype=int64))
14 (array([0, 1], dtype=int64), array([25341,   875], dtyp

- Train pipeline

In [91]:
pipeline.fit(X_train, Y_train)

Pipeline(steps=[('tfidf',
                 TfidfVectorizer(tokenizer=<function tokenize at 0x00000216624E90D0>)),
                ('mo',
                 MultiOutputClassifier(estimator=LogisticRegression(solver='liblinear')))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [92]:
Y_pred = pipeline.predict(X_validation)

Report the classification results

In [94]:
for i in range(0,N_CATEGORIES):
    print(classification_report(Y_validation[:,i], Y_pred[:,i]))

              precision    recall  f1-score   support

           0       0.91      0.98      0.94      4349
           1       0.83      0.53      0.65       895

    accuracy                           0.90      5244
   macro avg       0.87      0.75      0.80      5244
weighted avg       0.90      0.90      0.89      5244

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5218
           1       0.00      0.00      0.00        26

    accuracy                           1.00      5244
   macro avg       0.50      0.50      0.50      5244
weighted avg       0.99      1.00      0.99      5244

              precision    recall  f1-score   support

           0       0.79      0.87      0.83      3113
           1       0.78      0.66      0.71      2131

    accuracy                           0.78      5244
   macro avg       0.78      0.76      0.77      5244
weighted avg       0.78      0.78      0.78      5244

              preci

c:\Users\BLEPLAE\Anaconda3\envs\env2\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\BLEPLAE\Anaconda3\envs\env2\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\BLEPLAE\Anaconda3\envs\env2\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\BLEPLAE\An

              precision    recall  f1-score   support

           0       0.94      0.99      0.97      4774
           1       0.83      0.40      0.54       470

    accuracy                           0.94      5244
   macro avg       0.89      0.69      0.75      5244
weighted avg       0.93      0.94      0.93      5244

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      5171
           1       0.79      0.15      0.25        73

    accuracy                           0.99      5244
   macro avg       0.89      0.58      0.62      5244
weighted avg       0.99      0.99      0.98      5244

              precision    recall  f1-score   support

           0       0.98      1.00      0.99      5140
           1       0.55      0.06      0.10       104

    accuracy                           0.98      5244
   macro avg       0.76      0.53      0.55      5244
weighted avg       0.97      0.98      0.97      5244

              preci

c:\Users\BLEPLAE\Anaconda3\envs\env2\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\BLEPLAE\Anaconda3\envs\env2\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\BLEPLAE\Anaconda3\envs\env2\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\BLEPLAE\An

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = 

cv = 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
print(classification_report(y_true, y_pred))


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.